In [2]:
import sgt
sgt.__version__
from sgt import SGT

In [3]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

from scipy.optimize import minimize
from collections import OrderedDict

from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
# 값이 0 또는 1 이므로 로지스틱 회귀를 사용함

In [34]:
df = pd.read_csv("C:/Users/rltmdals/Downloads/user_action_10days.csv")

In [35]:
df_fillnan = df.fillna(999)
str_expr = "(day1 != 999) or \
    (day2 != 999) or \
    (day3 != 999) or \
    (day4 != 999) or \
    (day5 != 999) or \
    (day6 != 999) or \
    (day7 != 999) or \
    (day8 != 999) or \
    (day9 != 999) or \
    (day10 != 999)" 
df = df_fillnan.query(str_expr)
df.head()

,user_id,day1,day2,day3,day4,day5,day6,day7,day8,day9,day10,subscribe
0,fa2ad653-e7d9-4250-8db0-c62c962ccd3f,SMSMFEESMSMSSMSMJEFEFJSMSMSSMSMSMSFJESJFEJSGEH...,SMSMSJFESSMSEFEEJJFEI,999,999,SEJFSMSMSMSJJJJEFSFEJSMSMSSMFESMS,ELLLLLLLLLLLLKKKKKSMSMSEJFSJEFSMSMSSSSSMSMSEJF...,SMSMJFEEJFSMSMSSMSSMSMSEJFSFEJSMSMSSSMSMSEJFSF...,999,999,999,0
1,695dbd71-59ee-427e-aa64-4ff7afd1e519,E,999,999,999,999,999,999,999,999,999,0
2,695016e2-4c93-4d57-868e-a2b218519b6c,SMSMSSMSMSSMSMSMSFJKKKEKKKKSJFESMSMS,999,999,999,999,999,999,999,999,999,0
3,e7d6135d-3b19-4655-893d-23adb79d227b,IJJJJIJJJJJJJJIJJJLLJJJJIGJJFMMMEFEE,999,EKKKKKKLJFMMMEELELFELLEJJJJJJEJJIJLLLFIJJJJIJJ...,ELELLFEIJJIJJJJJJJJJJJJJLIJJJJJJIFEJFMMM,EJFMMEJFMMMEFILIFIILIIEIFIIIEEIG,999,EJGJ,999,999,999,0
4,6b8a2ed4-5eb7-4530-91ec-512c03907a1a,EFL,SMSSMSSMSMSSMSMSSMEFS,EFSMSMSSMSMSSMEFSMSMSMSSMFISSMLLELLLFSMFESMS,SMEFSSSMSSMFESMSMSMSSSMEFSSMEFSSMFESMSMSMSMSMS...,SSMSMSMSMSSMSMSMSMSSMSMSMSSMSMSMSSSSMSSSMFESMEFSS,SMSSMEJJJFSSMEFS,999,999,999,999,0


In [36]:
df['day2'] = df['day2'].apply(lambda x: 1 if x != 999 else 0)
df.head()

C:\Users\rltmdals\AppData\Local\Temp\ipykernel_21932\1987748044.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day2'] = df['day2'].apply(lambda x: 1 if x != 999 else 0)


,user_id,day1,day2,day3,day4,day5,day6,day7,day8,day9,day10,subscribe
0,fa2ad653-e7d9-4250-8db0-c62c962ccd3f,SMSMFEESMSMSSMSMJEFEFJSMSMSSMSMSMSFJESJFEJSGEH...,1,999,999,SEJFSMSMSMSJJJJEFSFEJSMSMSSMFESMS,ELLLLLLLLLLLLKKKKKSMSMSEJFSJEFSMSMSSSSSMSMSEJF...,SMSMJFEEJFSMSMSSMSSMSMSEJFSFEJSMSMSSSMSMSEJFSF...,999,999,999,0
1,695dbd71-59ee-427e-aa64-4ff7afd1e519,E,0,999,999,999,999,999,999,999,999,0
2,695016e2-4c93-4d57-868e-a2b218519b6c,SMSMSSMSMSSMSMSMSFJKKKEKKKKSJFESMSMS,0,999,999,999,999,999,999,999,999,0
3,e7d6135d-3b19-4655-893d-23adb79d227b,IJJJJIJJJJJJJJIJJJLLJJJJIGJJFMMMEFEE,0,EKKKKKKLJFMMMEELELFELLEJJJJJJEJJIJLLLFIJJJJIJJ...,ELELLFEIJJIJJJJJJJJJJJJJLIJJJJJJIFEJFMMM,EJFMMEJFMMMEFILIFIILIIEIFIIIEEIG,999,EJGJ,999,999,999,0
4,6b8a2ed4-5eb7-4530-91ec-512c03907a1a,EFL,1,EFSMSMSSMSMSSMEFSMSMSMSSMFISSMLLELLLFSMFESMS,SMEFSSSMSSMFESMSMSMSSSMEFSSMEFSSMFESMSMSMSMSMS...,SSMSMSMSMSSMSMSMSMSSMSMSMSSMSMSMSSSSMSSSMFESMEFSS,SMSSMEJJJFSSMEFS,999,999,999,999,0


In [5]:
sgt = SGT(flatten=False)
sequence = np.array(["B","B","A","C","A","C","A","A","B","A"])
sgt.fit(sequence)

,A,B,C
A,0.090616,0.131002,0.261849
B,0.086569,0.123042,0.052544
C,0.137142,0.028263,0.135335


In [6]:
sgt = SGT(flatten=True)
sequence = np.array(["B","B","A","C","A","C","A","A","B","A"])
sgt.fit(sequence)

(A, A)    0.090616
(A, B)    0.131002
(A, C)    0.261849
(B, A)    0.086569
(B, B)    0.123042
(B, C)    0.052544
(C, A)    0.137142
(C, B)    0.028263
(C, C)    0.135335
dtype: float64

In [7]:
corpus = pd.DataFrame([[1, ["B","B","A","C","A","C","A","A","B","A"]], 
                       [2, ["C", "Z", "Z", "Z", "D"]]], 
                      columns=['id', 'sequence'])
corpus

,id,sequence
0,1,"[B, B, A, C, A, C, A, A, B, A]"
1,2,"[C, Z, Z, Z, D]"


In [20]:
# SGT 모델이 쓸 수 있도록 데이터 조정
user_sequence_df= df[['user_id', 'day1']]

user_sequence_df.columns= ['id', 'sequence'] # SGT 모델에 입력시 컬럼명을 맞춰야 함

user_sequence_df['sequence']=user_sequence_df['sequence'].apply(lambda x: list(x))

user_sequence_df.head()

C:\Users\rltmdals\AppData\Local\Temp\ipykernel_21932\4226104426.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_sequence_df['sequence']=user_sequence_df['sequence'].apply(lambda x: list(x))


,id,sequence
0,fa2ad653-e7d9-4250-8db0-c62c962ccd3f,"[S, M, S, M, F, E, E, S, M, S, M, S, S, M, S, ..."
1,695dbd71-59ee-427e-aa64-4ff7afd1e519,[E]
2,695016e2-4c93-4d57-868e-a2b218519b6c,"[S, M, S, M, S, S, M, S, M, S, S, M, S, M, S, ..."
3,e7d6135d-3b19-4655-893d-23adb79d227b,"[I, J, J, J, J, I, J, J, J, J, J, J, J, J, I, ..."
4,6b8a2ed4-5eb7-4530-91ec-512c03907a1a,"[E, F, L]"


In [24]:
!pip install pandarallel

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pandarallel: filename=pandarallel-1.6.3-py3-none-any.whl size=16448 sha256=e7ccd0c999fd949a4104f4daf7dd0078b980ae7ac60baae9ed69af8d2e2d33e5
  Stored in directory: c:\users\rltmdals\appdata\local\pip\cache\wheels\66\69\3f\d563bce72ab17473f57a2c59ef94ee126a2dc5592f41ddf5e2
Successfully built pandarallel


In [25]:
import pandarallel

sgt = SGT(kappa=1, 
          flatten=True, 
          lengthsensitive=False,
          mode='multiprocessing')

sgt_df = sgt.fit_transform(user_sequence_df)
sgt_df.head()

INFO: Pandarallel will run on 19 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


,id,"(E, E)","(E, F)","(E, G)","(E, H)","(E, I)","(E, J)","(E, K)","(E, L)","(E, M)",...,"(S, E)","(S, F)","(S, G)","(S, H)","(S, I)","(S, J)","(S, K)","(S, L)","(S, M)","(S, S)"
0,fa2ad653-e7d9-4250-8db0-c62c962ccd3f,0.015076,0.028433,0.00845,0.053534,0.0,0.021605,0.003236,0.000000,0.004805,...,0.007577,0.008272,0.023151,0.003133,0.0,0.006758,0.000723,0.0,0.044276,0.013051
1,695dbd71-59ee-427e-aa64-4ff7afd1e519,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
2,695016e2-4c93-4d57-868e-a2b218519b6c,0.000335,0.000912,0.00000,0.000000,0.0,0.002479,0.142829,0.000000,0.038426,...,0.002507,0.026716,0.000000,0.000000,0.0,0.024976,0.001261,0.0,0.064473,0.025459
3,e7d6135d-3b19-4655-893d-23adb79d227b,0.184334,0.367879,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
4,6b8a2ed4-5eb7-4530-91ec-512c03907a1a,0.000000,0.367879,0.00000,0.000000,0.0,0.000000,0.000000,0.135335,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000


In [37]:
# 모델용 데이터 정리

X=sgt_df.iloc[:,1:101]
Y=df['day2']

# 훈련/테스트셋
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25,random_state=0)

# 모델 fitting(훈련용 데이터로 모델을 fitting함)

classifer=LogisticRegression(random_state=0)
classifer.fit(X_train,Y_train)

# 테스트 데이터를 fitting 한 모델에 넣어서 예측치 계산

Y_pred=classifer.predict(X_test)

# Confusion Matrix 로 평가

cmtx = pd.DataFrame(
    confusion_matrix(Y_test,Y_pred, labels=[1, 0]), 
    index=['실측:Comeback(1)', '실측:No Show(0)'], 
    columns=['예측:Comeback(1)', '예측:No Show(0)']
)

cmtx

C:\Users\rltmdals\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\rltmdals\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


,예측:Comeback(1),예측:No Show(0)
실측:Comeback(1),618,50
실측:No Show(0),312,62


In [38]:
accuracy_score(Y_test,Y_pred)

0.6525911708253359

In [39]:
precision_recall_fscore_support(Y_test,Y_pred, average='binary')

(0.6645161290322581, 0.9251497005988024, 0.7734668335419275, None)

In [40]:
# Day1 활동의 SGT 인코딩 X값들과, 전체 무료체험일 후 약정(전환) 여부에 대한 모델

X=sgt_df.iloc[:,1:101]
Y=df['subscribe']

# 훈련/테스트셋
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.25,random_state=0)

# 모델 fitting(훈련용 데이터로 모델을 fitting함)

classifer=LogisticRegression(random_state=0)
classifer.fit(X_train,Y_train)

# 테스트 데이터를 fitting 한 모델에 넣어서 예측치 계산

Y_pred=classifer.predict(X_test)

# Confusion Matrix 로 평가

cmtx = pd.DataFrame(
    confusion_matrix(Y_test,Y_pred, labels=[1, 0]), 
    index=['실측:subscribe(1)', '실측:no subscribe(0)'], 
    columns=['예측:subscribe(1)', '예측:no subscribe(0)']
)

cmtx

C:\Users\rltmdals\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\rltmdals\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


,예측:subscribe(1),예측:no subscribe(0)
실측:subscribe(1),1,229
실측:no subscribe(0),0,812


In [41]:
accuracy_score(Y_test,Y_pred)

0.7802303262955854

In [42]:
precision_recall_fscore_support(Y_test,Y_pred, average='binary')

(1.0, 0.004347826086956522, 0.008658008658008658, None)